In [202]:
import logging
import coloredlogs
import requests
from time import sleep

In [203]:
logging.basicConfig()
logger = logging.getLogger("poll-ooni")
# logger.setLevel(logging.DEBUG)
coloredlogs.install()
coloredlogs.install(level='DEBUG')
# coloredlogs.install(level='INFO')

In [204]:
from config import config
import psycopg2

# connect to the db
my_connection = psycopg2.connect(**config['postgres'])
my_cursor = my_connection.cursor()
logging.info('Connected to database.')

2021-05-25 15:02:31 congratulations root[4205] INFO Connected to database.


In [293]:
from datetime import datetime

def to_utc (t: datetime) -> datetime:
    return most_recent_reading.astimezone(pytz.utc)

In [5]:
# TODO make dry!
def is_nonempty_str(my_str: str) -> bool:
    return (type(my_str) == str) & (len(my_str) > 0)

class Alpha2 ():
    '''
    Represents an ISO alpha-2 country code.
    '''
    def __init__(self,
                 country_code: str):
        assert(is_nonempty_str(country_code))
        assert(len(country_code)==2)
        self.country_code = country_code
        
    def __str__(self):
        return f'{self.country_code}'
    
    def __repr__(self):
        return self.__str__()


# Getting the data we need

## Measuremnets from OONI

In [98]:
def api_query (query: str, results=[], queries=1, max_queries=None) -> list:
    '''Recursively query the API, up to `max_queries`. (If `max_queries=None`, we
    will paginate through the results as long as they run).
    '''
    base_url = 'https://api.ooni.io/api/v1/'
    query = '{!s}{!s}'.format(base_url, query)
    try:
        resp =  requests.get(query).json()
        results = results + resp['results']
        next_url = resp['metadata']['next_url']
        if max_queries is not None and queries > max_queries:
            return results
        if next_url:
            # sleep so as to not overwhelm the endpoint
            sleep(config['sleep-times']['ooni-paginate'])
            # remove base url to perfrom the query
            next_url = next_url.split('api/v1/')[1]
            print("next URL", next_url)
            return api_query(next_url, results, queries+1, max_queries)
        return results
    except Exception as inst:
        # if we have an error,
        logger.warning("Error querying API: {!s}".format(inst))
        # just return what we've collected
        # (at worst, `results` will be `[]`)
        return results


In [100]:
base_query = 'measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000'

def query_recent_measurements (max_queries=5) -> list:
    '''Queries all recent measurements, up to specified maximum number of queries.'''
    return api_query(base_query, max_queries=max_queries)

def get_blocking_type (measurement) -> str:
    '''Get blocking type, if available.'''
    try:
        return measurement['scores']['analysis']['blocking_type']
    except:
        return None

In [101]:
measurements = query_recent_measurements()

2021-05-25 13:19:23 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-05-25 13:19:24 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000 HTTP/1.1" 200 824369
2021-05-25 13:19:27 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443


next URL measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&offset=1000


2021-05-25 13:19:28 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&offset=1000 HTTP/1.1" 200 827717
2021-05-25 13:19:31 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443


next URL measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&offset=2000


2021-05-25 13:19:32 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&offset=2000 HTTP/1.1" 200 830892
2021-05-25 13:19:35 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443


...

2021-05-25 13:19:36 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&offset=3000 HTTP/1.1" 200 827264
2021-05-25 13:19:39 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443


next URL measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&offset=4000


2021-05-25 13:19:40 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&offset=4000 HTTP/1.1" 200 830291
2021-05-25 13:19:43 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443


next URL measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&offset=5000


2021-05-25 13:19:44 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&offset=5000 HTTP/1.1" 200 825740


In [102]:
len(measurements)

5995

In [12]:
m_detail = requests.get(m['measurement_url']).json()

m_detail['test_keys']['queries']
m_detail

2021-05-25 12:59:39 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): ams-pg.ooni.org:443
2021-05-25 12:59:39 congratulations urllib3.connectionpool[4205] DEBUG https://ams-pg.ooni.org:443 "GET /api/v1/raw_measurement?report_id=20210525T181651Z_webconnectivity_IN_55836_n1_v7tNboMf0kqxJ5gp&input=https%3A%2F%2Fwww.sendspace.com%2F HTTP/1.1" 200 10333


{'annotations': {'engine_name': 'ooniprobe-engine',
  'engine_version': '3.9.0',
  'flavor': 'stableFull',
  'network_type': 'wifi',
  'platform': 'android'},
 'data_format_version': '0.2.0',
 'input': 'https://www.sendspace.com/',
 'measurement_start_time': '2021-05-25 19:01:09',
 'probe_asn': 'AS55836',
 'probe_cc': 'IN',
 'probe_ip': '127.0.0.1',
 'probe_network_name': 'Reliance Jio Infocomm Limited',
 'report_id': '20210525T181651Z_webconnectivity_IN_55836_n1_v7tNboMf0kqxJ5gp',
 'resolver_asn': 'AS55836',
 'resolver_ip': '49.45.31.116',
 'resolver_network_name': 'Reliance Jio Infocomm Limited',
 'software_name': 'ooniprobe-android',
 'software_version': '2.11.0',
 'test_helpers': {'backend': {'address': 'https://wcth.ooni.io',
   'type': 'https'}},
 'test_keys': {'agent': 'redirect',
  'client_resolver': '49.45.31.116',
  'retries': None,
  'socksproxy': None,
  'network_events': [{'address': '172.64.100.30:443',
    'failure': None,
    'operation': 'connect',
    'proto': 'tcp',


## Get IP from URL

In [13]:
inputs = [m['input'] for m in measurements]

In [238]:
import socket
import urllib.parse
from typing import Optional

def get_hostname (url):
    return urllib.parse.urlparse(url).netloc

def fetch_ip_from_hostname (hostname: str) ->  str:
    try:
        return socket.gethostbyname(hostname)
    except Exception as inst:
            logger.warning(f"Error looking up IP of hostname {hostname}: {inst}")
            return None


In [245]:
my_ip = fetch_ip_from_hostname(get_hostname(inputs[505]))
my_ip

'172.67.145.82'

## Get geolocation from IP

In [246]:
import geoip2.database
from typing import Optional

def ip_to_alpha2 (ip: str) -> Optional[Alpha2]:
    with geoip2.database.Reader('dbip-country-lite-2021-05.mmdb') as reader:
        try:
            response = reader.country(ip)
            return Alpha2(response.country.iso_code)
        except Exception as inst:
            # if we have an error,
            logger.warning(f"Error looking up country code of IP {ip}: {inst}")
            return None
    
ip_to_alpha2(my_ip)

US

## Cacheing IP

### A data model for IP Hostname mapping

In [263]:
from IPy import IP
class IPHostnameMapping:
    def __init__ (
        self,
        ip: str,
        hostname: str,
        time: pd.Timestamp,
    ):
        assert(is_nonempty_str(hostname))
        self.hostname = hostname
        # this will throw if `ip` isn't valid
        ip = IP(ip)
        # check that it's a public IP!
        assert(ip.iptype() == 'PUBLIC')
        # we'll just save the IP as a string for simplicity's sake
        self.ip = ip.strNormal()
        assert(type(time) == pd.Timestamp)
        self.time = time
    
    def create_table(
            self,
            cur: cursor,
            conn: connection):
        cmd = """
          CREATE TABLE ip_hostname_mapping (
             hostname                  VARCHAR NOT NULL,
             ip                        VARCHAR NOT NULL,
             time                      TIMESTAMPTZ NOT NULL
          )
        """
        cur.execute(cmd)
        conn.commit()

    def write_to_db(
            self,
            cur: cursor,
            conn: connection,
            commit=True,
    ):
        cur.execute(
            """
            INSERT INTO ip_hostname_mapping
            (hostname, ip, time)
            VALUES
            (%s, %s, %s)
            """, (self.hostname,
                  self.ip,
                  self.time
                 ))
        if commit:
            return conn.commit()
        return

    def __str__(self):
        return f'{self.time: self.hostname->self.ip}'
    

In [264]:
ip = IP(my_ip)
ip.iptype()

'PUBLIC'

In [237]:
mapping = IPHostnameMapping('wikipedia.org', '198.35.26.96', now())
# mapping.create_table(my_cursor, my_connection)

In [227]:
get_ip('wikipedia.org')

'198.35.26.96'

## Integrate database read/writes into IP lookup

In [271]:
from typing import Tuple
def retrieve_ip (cur: cursor, hostname: str) -> Optional[Tuple[datetime, str]]:
    cur.execute('''
    SELECT time, ip from ip_hostname_mapping
    WHERE hostname=hostname
    ORDER BY time DESC
    ''')
    return cur.fetchone()

In [301]:
from datetime import timedelta

def lookup_ip (cur: cursor, conn: connection, hostname: str, 
               cache_expiry: timedelta = timedelta(days=1)):
    # if we have a result in our DB
    time_ip_tuple = retrieve_ip(cur, hostname)
    if time_ip_tuple:
        time, ip = time_ip_tuple
        # and that result is fresh enough
        is_expired = (now() - cache_expiry) > to_utc(time)
        if not is_expired:
            # return it
            return ip
    # otherwise
    # fetch IP with a query
    ip = fetch_ip_from_hostname(hostname)
    # write that mapping to the DB for the future
    mapping = IPHostnameMapping(ip, hostname, now())
    mapping.write_to_db(cur, conn)
    # return the IP
    return ip
    
lookup_ip(my_cursor, my_connection, 'wikipedia.org')

'198.35.26.96'

In [304]:
def url_to_alpha2 (cur: cursor, conn: connection, url: str) -> Optional[Alpha2]:
    hostname = get_hostname(url)
    maybe_ip = lookup_ip(cur, conn, hostname)
    if maybe_ip is None:
        return None
    return ip_to_alpha2(maybe_ip)

url_to_alpha2(my_cursor, my_connection, 'https://wikipedia.org/')

US

## Get TLD jurisdiction

In [554]:
from imp import reload
from src.w3techs import utils as w3techs_utils
reload(w3techs_utils)
from tldextract import extract

In [546]:
import idna

In [547]:
def get_tld_jurisdiction (url: str) -> Optional[Alpha2]:
    '''
    Takes a URL and gets an Alpha 2
    representing the jurisdiction of the URL's top-level domain.
    '''
    # sometimes, URL is an IP - that's fine
    hostname = get_hostname(url)
    try:
        IP(hostname) # if this doesn't throw an exception
        return None
    except:
        pass
    # decode IDNA (internationalized) hostnames
    # e.g. http://xn--80aaifmgl1achx.xn--p1ai/
    decoded_hostname = idna.decode(hostname)
    tld = extract(decoded_hostname)
    # get last item in url
    # e.g., '.com.br' should be '.br'
    tld = tld.suffix
    cc_tld = tld.split('.')[-1]
    # put it
    cc_tld_str = f'.{cc_tld}'
    cc =  w3techs_utils.get_country(cc_tld_str)
    if cc is not None:
        return Alpha2(cc)
    logger.warning(f'No jurisidiction for {tld} - {url}')
    return None

get_tld_jurisdiction('http://mycool.com.br')
get_tld_jurisdiction('https://1.1.1.1/dns-query?dns=q80BAAABAAAAAAAAA3d3dwdleGFtcGxlA2NvbQAAAQAB')
get_tld_jurisdiction('http://xn--80aaifmgl1achx.xn--p1ai/')

RU

# Model the datatype

In [30]:
import pandas as pd

def now () -> pd.Timestamp:
    return pd.Timestamp.utcnow()

def is_in_future (timestamp: pd.Timestamp) -> bool:
    return timestamp > now()

is_in_future(  pd.Timestamp('2021-06-24T20:36:06Z'))

True

In [558]:
from psycopg2.extensions import cursor
from psycopg2.extensions import connection
import pandas as pd

class OONIWebConnectivityTest():
    '''
    Class to capture results of an OONI web connectivity test.
      - https://ooni.org/nettest/web-connectivity/
    See README for more details on these fields.
    
    This is where validation happens.
    TODO - Check for SQL injection attacks.
    '''
    def __init__(self,
                  blocking_type: str,
                  probe_alpha2: Alpha2,
                  input_url: str,
                  anomaly: bool,
                  confirmed: bool,
                  report_id: str,
                  input_ip_alpha2: Alpha2,
                  tld_jurisdiction_alpha2: Alpha2,
                  measurement_start_time: pd.Timestamp):
            # we only want stuff where blocking actually happened 
            assert(blocking_type != False)
            self.blocking_type = blocking_type
            
            assert(type(probe_alpha2) == Alpha2)
            self.probe_alpha2 = str(probe_alpha2)
            
            assert(is_nonempty_str(input_url))
            self.input_url = input_url
            
            assert(type(anomaly) == bool)
            self.anomaly = anomaly
            
            assert(type(confirmed) == bool)
            self.confirmed = confirmed
            
            assert(is_nonempty_str(report_id))
            self.report_id = report_id
            
            # type is optional
            assert((type(input_ip_alpha2) == Alpha2) or 
                   (input_ip_alpha2 == None))
            if input_ip_alpha2 == None:
                self.input_ip_alpha2 = None
            else:
                self.input_ip_alpha2 = str(input_ip_alpha2)
            
            # type is optional
            assert((type(tld_jurisdiction_alpha2) == Alpha2) or
                   (tld_jurisdiction_alpha2 == None))
            if tld_jurisdiction_alpha2 == None:
                self.tld_jurisdiction_alpha2 = None
            else:
                self.tld_jurisdiction_alpha2 = str(tld_jurisdiction_alpha2)
            
            assert(type(measurement_start_time) == pd.Timestamp)
            # if the timestamp is in the future...
            if is_in_future(measurement_start_time):
                logging.debug(f'Time is in future: {measurement_start_time}. Setting time to now.')
                # set the time to now.
                self.measurement_start_time = now()
            # otherwise
            else:
                # set it to whenever it was reported
                self.measurement_start_time = measurement_start_time
            
    def create_table(
            self,
            cur: cursor,
            conn: connection):
        cmd = """
          CREATE TABLE ooni_web_connectivity_test (
             blocking_type             VARCHAR,
             probe_alpha2              CHAR(2) NOT NULL,
             input_url                 VARCHAR NOT NULL,
             anomaly                   BOOLEAN NOT NULL,
             confirmed                 BOOLEAN NOT NULL,
             report_id                 VARCHAR NOT NULL,
             input_ip_alpha2           CHAR(2),
             tld_jurisdiction_alpha2   CHAR(2),
             measurement_start_time    TIMESTAMPTZ NOT NULL
          )
        """
        cur.execute(cmd)
        conn.commit()

    def write_to_db(
            self,
            cur: cursor,
            conn: connection,
            commit=True,
    ):
        cur.execute(
            """
            INSERT INTO ooni_web_connectivity_test
            (blocking_type, probe_alpha2, input_url, anomaly, confirmed, report_id,  
            input_ip_alpha2, tld_jurisdiction_alpha2, measurement_start_time)   
            VALUES
            (%s, %s, %s, %s, %s, %s, %s, %s, %s)   
            """, (self.blocking_type,
                  self.probe_alpha2,
                  self.input_url,
                  self.anomaly,
                  self.confirmed,
                  self.report_id,
                  self.input_ip_alpha2,
                  self.tld_jurisdiction_alpha2,
                  self.measurement_start_time))
        if commit:
            return conn.commit()
        return

    def __str__(self):
        # TODO make DRY with write_to_db?
        # TODO do this in general?
        return f'{self.measurement_start_time} - {self.probe_alpha2} -> {self.input_ip_alpha2}, {self.tld_jurisdiction_alpha2} ({self.blocking_type} {self.input_url})'

    def __repr__(self):
        return self.__str__()


# Marshall from datatype

In [339]:
def ingest_api_measurement (measurement: dict) -> OONIWebConnectivityTest:
    # make a connection and cursor (for this thread)
    connection = psycopg2.connect(**config['postgres'])
    cursor = connection.cursor()
    blocking_type = get_blocking_type(measurement)
    probe_alpha2 = Alpha2(measurement['probe_cc'])
    input_url = measurement['input']
    anomaly = measurement['anomaly']
    confirmed = measurement['confirmed']
    report_id = measurement['report_id']
    input_ip_alpha2 = url_to_alpha2(cursor, connection, input_url) 
    tld_jurisdiction_alpha2 = get_tld_jurisdiction(input_url)
    measurement_start_time = pd.Timestamp(measurement['measurement_start_time'])
    return OONIWebConnectivityTest(
        blocking_type,
        probe_alpha2,
        input_url,
        anomaly,
        confirmed,
        report_id,
        input_ip_alpha2,
        tld_jurisdiction_alpha2,
        measurement_start_time
    )

ingest_api_measurement(measurements[0])

2021-05-25 20:19:18+00:00 - CA -> US, US (http-diff http://www.khilafah.com/)

In [519]:
from multiprocessing import Pool
from typing import List

def ingest_api_measurements (measurements: List[dict]) -> List[OONIWebConnectivityTest]:
    with Pool() as p:
        return p.map(ingest_api_measurement, measurements)

# Test it with the DB

In [35]:
# dummy data
OONIWebConnectivityTest(
    'example',
    Alpha2('US'),
    'example',
    False,
    False,
    'example',
    Alpha2('US'),
    Alpha2('US'),
    pd.Timestamp('2000-01-01 21:41:37+00:00')
).create_table(my_cursor, my_connection)

ProgrammingError: relation "ooni_web_connectivity_test" already exists


In [36]:
def rollback():
    my_cursor.execute("ROLLBACK")
    my_connection.commit()

In [42]:
rollback()
# test = None
# for mt in my_types:
#     try:
#         mt.write_to_db(my_cursor, my_connection, commit=False) 
#     except Exception as e:
#         test = mt
#         print(mt)
#         print(e)
#         break
        

In [38]:
# my_connection.commit()

# Set up a flow

In [552]:
from datetime import datetime

def get_latest_reading_time (cur: cursor) -> datetime:
    '''Get time of most recent measurement'''
    cur.execute('SELECT measurement_start_time from ooni_web_connectivity_test ORDER BY measurement_start_time DESC')
    return cur.fetchone()[0]
    
most_recent_reading = get_latest_reading_time(my_cursor)
most_recent_reading

datetime.datetime(2021, 5, 25, 18, 20, 20, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-420, name=None))

In [523]:
import pytz

def query_measurements_after (time: datetime) -> list:
    '''Queries all measurements after time.'''

    def fmt_dt (t: datetime):
        return t.strftime("%Y-%m-%dT%H:%M:%S")
    # format timezone-aware date into UTC fo querying
    utc_dt = to_utc(time)
    # format it into the query url
    dt_str = fmt_dt(utc_dt)
    query_str = base_query + f'&since={dt_str}'
    # issue the query
    return api_query(query_str)

ms  = query_measurements_after(most_recent_reading)

2021-05-25 16:22:33 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-05-25 16:22:33 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-25T23:21:39 HTTP/1.1" 200 1901


In [524]:
ingest_api_measurement(ms[0])

2021-05-25 23:22:11+00:00 - US -> US, US (http-diff http://www.newnownext.com/franchise/the-backlot/)

In [542]:
def write_to_db (cur: cursor, conn: connection, connectivity_tests: List[OONIWebConnectivityTest]) -> None:
    for t in connectivity_tests:
        t.write_to_db(cur, conn, commit=False) 
    conn.commit()

# Tying it all together

In [543]:
def scrape (cur: cursor, conn: connection) -> None:
    t = get_latest_reading_time(cur)
    logging.info(f'Querying results after {t}.')
    ms = query_measurements_after(t)
    logging.debug(f'Retrieved {len(ms)} results.')
    ingested = ingest_api_measurements(ms)
    logging.debug(f'Ingested {len(ingested)} results.')
    write_to_db(cur, conn, ingested)
    logging.info(f'Wrote {len(ingested)} results to database.')

In [544]:
scrape(my_cursor, my_connection)

2021-05-25 18:18:02 congratulations root[4205] INFO Querying results after 2021-05-25 18:13:14-07:00.
2021-05-25 18:18:02 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-05-25 18:18:03 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-25T23:24:20 HTTP/1.1" 200 845524
2021-05-25 18:18:08 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443


next URL measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-25T23%3A24%3A20&offset=1000


2021-05-25 18:18:09 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-25T23%3A24%3A20&offset=1000 HTTP/1.1" 200 838261
2021-05-25 18:18:14 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443


next URL measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-25T23%3A24%3A20&offset=2000


2021-05-25 18:18:15 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-25T23%3A24%3A20&offset=2000 HTTP/1.1" 200 421139
2021-05-25 18:18:17 congratulations root[4205] DEBUG Retrieved 2507 results.
2021-05-25 18:18:17 congratulations root[19887] INFO Cannot find country for .as
2021-05-25 18:18:17 congratulations poll-ooni[19887] WARNING No jurisidiction for as - https://write.as/
2021-05-25 18:18:19 congratulations root[19888] INFO Cannot find country for .fund
2021-05-25 18:18:19 congratulations poll-ooni[19888] WARNING No jurisidiction for fund - https://www.opentech.fund/
2021-05-25 18:18:19 congratulations root[19887] INFO Cannot find country for .jm
2021-05-25 18:18:19 congratulations poll-ooni[19887] WARNING No jurisidiction for org.jm - https://www.isa.org.jm/
2021-05-25 18:18:21 congratulations root[19886] INFO Country code for .mobi is not

KeyboardInterrupt: 

In [ ]:
while True:
    scrape(mycursor, my_connection)
    sleep(config['sleep-times']['ooni-poll'])

2021-05-25 19:08:29 congratulations root[4205] INFO Querying results after 2021-05-25 19:05:08-07:00.
2021-05-25 19:08:29 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-05-25 19:08:29 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-26T01:20:20 HTTP/1.1" 200 314453
2021-05-25 19:08:30 congratulations root[4205] DEBUG Retrieved 379 results.
2021-05-25 19:08:32 congratulations root[4205] DEBUG Ingested 379 results.
2021-05-25 19:08:32 congratulations root[4205] INFO Wrote 379 results to database.
2021-05-25 19:13:32 congratulations root[4205] INFO Querying results after 2021-05-25 19:08:01-07:00.
2021-05-25 19:13:32 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-05-25 19:13:33 congratulations urllib3.connectionpool[4205] DEBUG htt

2021-05-25 19:59:19 congratulations root[7482] DEBUG Time is in future: {measurement_start_time}. Setting time to now.
2021-05-25 19:59:19 congratulations root[7482] DEBUG Time is in future: {measurement_start_time}. Setting time to now.
2021-05-25 19:59:21 congratulations root[4205] DEBUG Ingested 795 results.
2021-05-25 19:59:21 congratulations root[4205] INFO Wrote 795 results to database.
2021-05-25 20:04:22 congratulations root[4205] INFO Querying results after 2021-05-25 19:59:19.298985-07:00.
2021-05-25 20:04:22 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-05-25 20:04:23 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-26T01:20:20 HTTP/1.1" 200 746353
2021-05-25 20:04:26 congratulations root[4205] DEBUG Retrieved 900 results.
2021-05-25 20:04:26 congratulations root[8421] DEBUG 

next URL measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-26T01%3A20%3A20&offset=1000


2021-05-25 20:09:36 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-26T01%3A20%3A20&offset=1000 HTTP/1.1" 200 7649
2021-05-25 20:09:36 congratulations root[4205] DEBUG Retrieved 1009 results.
2021-05-25 20:09:36 congratulations root[9446] DEBUG Time is in future: {measurement_start_time}. Setting time to now.
2021-05-25 20:09:36 congratulations root[9446] DEBUG Time is in future: {measurement_start_time}. Setting time to now.
2021-05-25 20:09:36 congratulations root[9446] DEBUG Time is in future: {measurement_start_time}. Setting time to now.
2021-05-25 20:09:36 congratulations root[9446] DEBUG Time is in future: {measurement_start_time}. Setting time to now.
2021-05-25 20:09:36 congratulations root[9446] DEBUG Time is in future: {measurement_start_time}. Setting time to now.
2021-05-25 20:09:39 congratulations root[4205] DEBUG Ingested 1009 res

next URL measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-26T01%3A20%3A20&offset=1000


...

2021-05-26 08:44:48 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-26T01%3A20%3A20&offset=13000 HTTP/1.1" 200 823804
2021-05-26 08:44:53 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443


next URL measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-26T01%3A20%3A20&offset=14000


2021-05-26 08:44:54 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-26T01%3A20%3A20&offset=14000 HTTP/1.1" 200 825519
2021-05-26 08:44:59 congratulations urllib3.connectionpool[4205] DEBUG Starting new HTTPS connection (1): api.ooni.io:443


next URL measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-26T01%3A20%3A20&offset=15000


2021-05-26 08:45:00 congratulations urllib3.connectionpool[4205] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-26T01%3A20%3A20&offset=15000 HTTP/1.1" 200 458513
2021-05-26 08:45:02 congratulations root[4205] DEBUG Retrieved 15550 results.
2021-05-26 08:45:02 congratulations root[17709] INFO Cannot find country for .red
2021-05-26 08:45:02 congratulations poll-ooni[17709] WARNING No jurisidiction for red - https://paratodasnos.red/
2021-05-26 08:45:03 congratulations root[17709] INFO Cannot find country for .re
2021-05-26 08:45:03 congratulations poll-ooni[17709] WARNING No jurisidiction for re - http://myfreemp3.re/
2021-05-26 08:45:05 congratulations root[17709] INFO Country code for .mobi is not a string: nan
2021-05-26 08:45:05 congratulations poll-ooni[17709] WARNING No jurisidiction for mobi - http://desiden.mobi/
2021-05-26 08:45:09 congratulations root[17710] INFO Country code for